### Carrega libs

In [1]:
# scrape data
from bs4 import BeautifulSoup as bs
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

# dataframe
import pandas as pd
import os

### Kabum

In [2]:
def convert_price_to_float(price_str):
    # Checa de price_str é None ou se contém '----'
    if not price_str or '----' in price_str:
        return None
    try:
        # Remove R$
        price_str = price_str.replace('R$', '').strip()
        # Remove separador de milhar
        price_str = price_str.replace('.', '')
        # Troca , por .
        price_str = price_str.replace(',', '.')
        # Converte pra float
        price_float = float(price_str)
        return price_float
    except ValueError:
        return None

In [3]:
def extract_especifications(category, name):
    if category == 'disco-rigido-hd':
        # Define o padrão regex para encontrar o armazenamento (número seguido de TB ou GB)
        padrao = re.compile(r'\d+(?:TB|GB)', re.IGNORECASE)
        
        # Procura o padrão no nome do produto
        match = padrao.search(name)
        
        # Se encontrar, retorna o match; caso contrário, retorna None
        if match:
            return match.group()
        else:
            return None
    else:
        return None

In [22]:
def clean_item_name(name):
    # Remove " , ' , ` , ´
    cleaned_name = re.sub(r'[\"\`\´\']', '', name)
    cleaned_name = re.sub(r'-+', '-', cleaned_name)
    cleaned_name = cleaned_name.strip('-')
    return cleaned_name


In [5]:
def scrape_data_kabum(URL):
    r = requests.get(URL)
    soup = bs(r.content)

    # Número total de páginas
    NUM_PAGES = int(((soup.find('li', class_='next')).find_previous_sibling('li')).text.strip())

    # Configurações de Driver
    driver = webdriver.Chrome()
    driver.get(URL)

    # loop de págians
    items = []
    
    # URL.split('?')[0]: divide a URL pelo caractere '?' e pega a primeira parte antes dos parâmetros de consulta.
    # .split('/'): divide a parte restante da URL pelo caractere '/' 
    # [3]: Acessa o quarto elemento da lista, que é a categoria
    CATEGORIA = ((URL.split('?')[0]).split('/'))[3]
    
    # [-1]: Acessa o último elemento da lista, que é a subcategoria
    SUBCATEGORIA = ( ( (URL.split('?'))[0] ).split('/') )[-1]

    PAGE = 1
    while PAGE <= NUM_PAGES:
        print(f"Processando página {PAGE} de {NUM_PAGES} - kabum/{CATEGORIA}/{SUBCATEGORIA}")
        wait = WebDriverWait(driver, 10)
        
        # Aceitar cookies - Necessário pois o componente sobrepõe itens da página
        try:
            entendi_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
            entendi_button.click()
        except:
            pass
        
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article.productCard')))
        articles = driver.find_elements(By.CSS_SELECTOR, 'article.productCard')
        
        for article in articles:
            item_url = ('https://www.kabum.com.br/' + article.find_element(By.CSS_SELECTOR, 'a.productLink').get_attribute('href') )
            if item_url:
                item_url = item_url
            else:
                item_url = None 
                
            item_name = article.find_element(By.CSS_SELECTOR, 'h3').text
            #item_name = clean_item_name(item_name)
            
            item_image = article.find_element(By.CSS_SELECTOR, 'a.productLink > img').get_attribute('src')
            item_openBox = article.find_elements(By.CLASS_NAME, 'openboxTagCard')
            if item_openBox:
                item_openBox = True
            else:
                item_openBox = False
                
            item_price = article.find_element(By.CLASS_NAME, 'priceCard').text
            item_price = convert_price_to_float(item_price)   
            
            item_especifications = extract_especifications(SUBCATEGORIA, item_name) 
                
            items.append({
                'categoria': CATEGORIA,
                'subcategoria': SUBCATEGORIA,
                'nome': item_name,
                'preço': item_price,
                'info': item_especifications,
                'openBox': item_openBox,
                'imagem': item_image,
                'site': 'kabum',
                'url': item_url,
                'data': datetime.now().strftime("%d-%m-%Y %H:%M:%S")
            })
            
        try:
            next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.next a.nextLink')))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            driver.execute_script("arguments[0].click();", next_button)
            PAGE += 1
        except Exception as e:
            print("Erro ao clicar no botão próximo:", e)
            break
    driver.quit()
    return items

### Tratamento dos dados

In [6]:
links = [
    'https://www.kabum.com.br/hardware/disco-rigido-hd?page_number=1&page_size=100&facet_filters=eyJJbnRlcmZhY2UiOlsiU0FUQSJdfQ==&sort=most_searched',
    'https://www.kabum.com.br/hardware/placa-de-video-vga?page_number=1&page_size=10000&facet_filters=&sort=most_searched'
]

In [7]:
produtos = []
for link in links:
    produtos.append(scrape_data_kabum(link))

Processando página 1 de 1 - kabum/hardware/disco-rigido-hd
Processando página 1 de 7 - kabum/hardware/placa-de-video-vga
Processando página 2 de 7 - kabum/hardware/placa-de-video-vga
Processando página 3 de 7 - kabum/hardware/placa-de-video-vga
Processando página 4 de 7 - kabum/hardware/placa-de-video-vga
Processando página 5 de 7 - kabum/hardware/placa-de-video-vga
Processando página 6 de 7 - kabum/hardware/placa-de-video-vga
Processando página 7 de 7 - kabum/hardware/placa-de-video-vga


Cria dataframe e CSV

In [8]:
dfs = []
for produto in produtos:
    df = pd.DataFrame(produto)
    dfs.append(df)

df_kabum = pd.concat(dfs, ignore_index=True)

In [23]:
df_kabum['nome'] = df_kabum['nome'].apply(clean_item_name)
df_kabum['subcategoria'] = df_kabum['subcategoria'].apply(clean_item_name)

In [27]:
df_kabum

,categoria,subcategoria,nome,preço,info,openBox,imagem,site,url,data
0,hardware,disco-rigido-hd,"HD Seagate 2TB BarraCuda, 3.5, SATA - ST2000DM008",419.99,2TB,False,https://images.kabum.com.br/produtos/fotos/100...,kabum,https://www.kabum.com.br/https://www.kabum.com...,10-06-2024 10:32:12
1,hardware,disco-rigido-hd,"HD Seagate 4TB BarraCuda, 3.5, SATA - ST4000DM004",639.99,4TB,False,https://images.kabum.com.br/produtos/fotos/958...,kabum,https://www.kabum.com.br/https://www.kabum.com...,10-06-2024 10:32:12
2,hardware,disco-rigido-hd,"HD Western Digital Purple, 2TB, 64mb, Sata 3 -...",429.99,2TB,False,https://images.kabum.com.br/produtos/fotos/472...,kabum,https://www.kabum.com.br/https://www.kabum.com...,10-06-2024 10:32:12
3,hardware,disco-rigido-hd,"HD WD Red Plus, 4TB, 5400 RPM, 3.5, SATA - WD4...",699.99,4TB,False,https://images.kabum.com.br/produtos/fotos/460...,kabum,https://www.kabum.com.br/https://www.kabum.com...,10-06-2024 10:32:12
4,hardware,disco-rigido-hd,"HD WD Red Plus, 6TB, 5400 RPM, 3.5, SATA - WD6...",999.99,6TB,False,https://images.kabum.com.br/produtos/fotos/460...,kabum,https://www.kabum.com.br/https://www.kabum.com...,10-06-2024 10:32:13
...,...,...,...,...,...,...,...,...,...,...
706,hardware,placa-de-video-vga,Placa de Vídeo RTX 3090 Xtreme Waterforce Giga...,NaN,None,False,https://images.kabum.com.br/produtos/fotos/167...,kabum,https://www.kabum.com.br/https://www.kabum.com...,10-06-2024 10:34:21
707,hardware,placa-de-video-vga,Placa de Vídeo Gigabyte Aorus Radeon RX 6700 X...,NaN,None,False,https://images.kabum.com.br/produtos/fotos/167...,kabum,https://www.kabum.com.br/https://www.kabum.com...,10-06-2024 10:34:21
708,hardware,placa-de-video-vga,Placa de Vídeo RX 6700 XT Sapphire Pulse AMD R...,NaN,None,False,https://images.kabum.com.br/produtos/fotos/217...,kabum,https://www.kabum.com.br/https://www.kabum.com...,10-06-2024 10:34:22
709,hardware,placa-de-video-vga,Placa de Vídeo Asus Rog Strix RTX 3080 10G V2 ...,NaN,None,False,https://images.kabum.com.br/produtos/fotos/249...,kabum,https://www.kabum.com.br/https://www.kabum.com...,10-06-2024 10:34:22


## Export to CSV

In [12]:
# Verificar se a pasta "data" existe, caso contrário, criá-la
if not os.path.exists('data'):
    os.makedirs('data')
    
# Caminho do arquivo CSV
csv_path = os.path.join('data', 'produtos_kabum.csv')

# Verificar se o arquivo já existe
file_exists = os.path.isfile(csv_path)

# Salvar o dataframe final em um arquivo CSV na pasta "data"
df_kabum.to_csv(csv_path, mode='a', index=False, header=not file_exists)